In [2]:
# 图像拍摄
import cv2
import numpy as np
import os
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
save_dir = os.path.join(os.getcwd(),"calibration","data")
print(save_dir)
num = 0
while True:
    ret, frame = cap.read()
    h,w,_ = frame.shape
    frame = frame[h-1::-1,::-1,::]
    # for i in range(0,480,20):
    #     cv2.line(frame,(0,i),(1280,i),(0,0,255),1)
    cv2.imshow("A video", frame)

    c = cv2.waitKey(1)
    # 按 Esc 退出
    if c == 27:
        break
    # 按 g 拍摄
    elif c == 32:
        img_1 = frame[:,0:640,:]
        img_2 = frame[:,640:1280,:]
        num  = num + 1
        name1 = f"./data/imgL_{num}.png"
        name2 = f"./data/imgR_{num}.png"

    
        cv2.imwrite(name1,img_1)
      
        cv2.imwrite(name2,img_2)
cap.release()
cv2.destroyAllWindows()

c:\Users\klay\Downloads\mobile_tof-2\calibration\calibration\data


KeyboardInterrupt: 

In [2]:
def save_coefficients(mtx, dist, path):
    """ Save the camera matrix and the distortion coefficients to given path/file. """
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write("K", mtx)
    cv_file.write("D", dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()


def save_stereo_coefficients(path, K1, D1, K2, D2, R, T, E, F, R1=None, R2=None, P1=None, P2=None, Q=None):
    """ Save the stereo coefficients to given path/file. """
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write("K1", K1)
    cv_file.write("D1", D1)
    cv_file.write("K2", K2)
    cv_file.write("D2", D2)
    cv_file.write("R", R)
    cv_file.write("T", T)
    cv_file.write("E", E)
    cv_file.write("F", F)
    cv_file.write("R1", R1)
    cv_file.write("R2", R2)
    cv_file.write("P1", P1)
    cv_file.write("P2", P2)
    cv_file.write("Q", Q)
    cv_file.release()


def load_coefficients(path):
    """ Loads camera matrix and distortion coefficients. """
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode("K").mat()
    dist_matrix = cv_file.getNode("D").mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]


def load_stereo_coefficients(path):
    """ Loads stereo matrix coefficients. """
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    K1 = cv_file.getNode("K1").mat()
    D1 = cv_file.getNode("D1").mat()
    K2 = cv_file.getNode("K2").mat()
    D2 = cv_file.getNode("D2").mat()
    R = cv_file.getNode("R").mat()
    T = cv_file.getNode("T").mat()
    E = cv_file.getNode("E").mat()
    F = cv_file.getNode("F").mat()
    R1 = cv_file.getNode("R1").mat()
    R2 = cv_file.getNode("R2").mat()
    P1 = cv_file.getNode("P1").mat()
    P2 = cv_file.getNode("P2").mat()
    Q = cv_file.getNode("Q").mat()

    cv_file.release()
    # return [K1, D1, K2, D2, R, T, E, F]
    return [K1, D1, K2, D2, R, T, E, F, R1, R2, P1, P2, Q]



In [4]:
# 单相机棋盘格标定
import cv2
import glob
import numpy as np
from tqdm import tqdm
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
params = cv2.SimpleBlobDetector_Params()
params.maxArea = 5e2
params.minArea = 100
params.minDistBetweenBlobs = 10

blobDetector = cv2.SimpleBlobDetector_create(params)


def calibrate(dirpath, prefix, image_format, square_size, width=9, height=6, board_type="chessboard"):
    """ Apply camera calibration operation for images in the given directory path. """
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size  # Create real world coords. Use your metric.

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    # Directory path correction. Remove the last character if it is '/'
    if dirpath[-1:] == '/':
        dirpath = dirpath[:-1]

    # Get the images
    images = glob.glob(dirpath+'/' + prefix + '*.' + image_format)

    # Iterate through the pairs and find chessboard corners. Add them to arrays
    # If openCV can't find the corners in an image, we discard the image.
    for fname in tqdm(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        keypoints = blobDetector.detect(gray)
        circle_img = img.copy()
        for kp in keypoints:
            cv2.circle(circle_img, (int(kp.pt[0]), int(kp.pt[1])), 5, (0, 0, 255), 3)
        if board_type == "chessboard":
            ret, corners = cv2.findChessboardCorners(gray, (width, height), None)
        elif board_type == "circle":
            ret, corners = cv2.findCirclesGrid(gray, (width, height), cv2.CALIB_CB_ASYMMETRIC_GRID
                , blobDetector, None)
        else:
            raise NotImplementedError(f"{board_type} is not implemented")
        if ret:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

    flag = 0
    flag |= cv2.CALIB_USE_INTRINSIC_GUESS
    flag |= cv2.CALIB_ZERO_TANGENT_DIST
    # flag |= cv2.CALIB_FIX_K1
    # flag |= cv2.CALIB_FIX_K2
    flag |= cv2.CALIB_FIX_K3
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],np.asarray([0.,0.,0.,0.,0.]),flag)
    # use distortion parameter
    return ret, mtx, dist, rvecs, tvecs

image_dir = "./data_rs_stereo/"
ret, mtx, dist, rvecs, tvecs = calibrate(image_dir,"imgL", "png", 0.35, 9, 6)
save_coefficients(mtx, dist, "stereo_R.yml")
ret, mtx, dist, rvecs, tvecs = calibrate(image_dir,"imgR", "png", 0.35, 9, 6)
save_coefficients(mtx, dist, "rs.yml")

100%|██████████| 18/18 [00:00<00:00, 26.77it/s]


In [19]:
# 双目立体标定
import numpy as np
import cv2 as cv2
import glob
import argparse
import sys

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
image_size = None

def stereo_calibrate(left_file, right_file, left_dir, left_prefix, right_dir, right_prefix, image_format, save_file, square_size,
        width=9, height=6):
    """ Stereo calibration and rectification """
    objp, leftp, rightp = load_image_points(left_dir, left_prefix, right_dir, right_prefix, image_format,
         square_size, width, height)

    K1, D1 = load_coefficients(left_file)
    K2, D2 = load_coefficients(right_file)
    flag = 0
    flag |= cv2.CALIB_ZERO_DISPARITY
    flag |= cv2.CALIB_USE_INTRINSIC_GUESS
    flag |= cv2.CALIB_ZERO_TANGENT_DIST
    # flag |= cv2.CALIB_FIX_K2
    flag |= cv2.CALIB_FIX_K3
    flag |= cv2.CALIB_FIX_K4
    flag |= cv2.CALIB_FIX_K5
    flag |= cv2.CALIB_FIX_K6

    # flag |= cv2.CALIB_FIX_INTRINSIC
    # flag |= cv2.CALIB_USE_INTRINSIC_GUESS
    # flag |= cv2.CALIB_FIX_FOCAL_LENGTH
    # flag |= cv2.CALIB_ZERO_TANGENT_DIST

    ret, K1, D1, K2, D2, R, T, E, F = cv2.stereoCalibrate(objp, leftp, rightp, K1, D1, K2, D2, image_size,flags=flag,criteria=criteria)
    print("Stereo calibration rms: ", ret)
    # MARK: remove distortion
    # D1 = D2 = np.asarray([0,0,0,0,0])

    R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(K1,D1, K2,D2, image_size, R, T, flags=cv2.CALIB_ZERO_DISPARITY)

    # save_stereo_coefficients(save_file, K1, D1, K2, D2, R, T, E, F)
    save_stereo_coefficients(save_file, K1, D1, K2, D2, R, T, E, F, R1, R2, P1, P2, Q)


def load_image_points(left_dir, left_prefix, right_dir, right_prefix, image_format, square_size, width=9, height=6):
    global image_size
    pattern_size = (width, height)  # Chessboard size!
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height * width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size  # Create real world coords. Use your metric.

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    left_imgpoints = []  # 2d points in image plane.
    right_imgpoints = []  # 2d points in image plane.

    # Left directory path correction. Remove the last character if it is '/'
    if left_dir[-1:] == '/':
        left_dir = left_dir[:-1]

    # Right directory path correction. Remove the last character if it is '/'
    if right_dir[-1:] == '/':
        right_dir = right_dir[:-1]

    # Get images for left and right directory. Since we use prefix and formats, both image set can be in the same dir.
    left_images = glob.glob(left_dir + '/' + left_prefix + '*.' + image_format)
    right_images = glob.glob(right_dir + '/' + right_prefix + '*.' + image_format)

    # Images should be perfect pairs. Otherwise all the calibration will be false.
    # Be sure that first cam and second cam images are correctly prefixed and numbers are ordered as pairs.
    # Sort will fix the globs to make sure.
    left_images.sort()
    right_images.sort()

    # Pairs should be same size. Otherwise we have sync problem.
    if len(left_images) != len(right_images):
        print("Numbers of left and right images are not equal. They should be pairs.")
        print("Left images count: ", len(left_images))
        print("Right images count: ", len(right_images))
        sys.exit(-1)

    pair_images = zip(left_images, right_images)  # Pair the images for single loop handling

    # Iterate through the pairs and find chessboard corners. Add them to arrays
    # If openCV can't find the corners in one image, we discard the pair.
    num = 0
    for left_im, right_im in pair_images:
        if num>1:
            continue
        num+=1
        # Right Object Points
        right = cv2.imread(right_im)
        gray_right = cv2.cvtColor(right, cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret_right, corners_right = cv2.findChessboardCorners(gray_right, pattern_size,
                                                             cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)
        # Left Object Points
        left = cv2.imread(left_im)
        gray_left = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret_left, corners_left = cv2.findChessboardCorners(gray_left, pattern_size,
                                                           cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FILTER_QUADS)

        if ret_left and ret_right:  # If both image is okay. Otherwise we explain which pair has a problem and continue
            # Object points
            objpoints.append(objp)
            # Right points
            corners2_right = cv2.cornerSubPix(gray_right, corners_right, (5, 5), (-1, -1), criteria)
            right_imgpoints.append(corners2_right)
            # Left points
            corners2_left = cv2.cornerSubPix(gray_left, corners_left, (5, 5), (-1, -1), criteria)
            left_imgpoints.append(corners2_left)
            l_c = np.asanyarray(corners_left).reshape([54,2])
            r_c = np.asanyarray(corners_right).reshape([54,2])
            l_c = np.sort(l_c,0)
            l_c = np.sort(l_c,1)
            r_c = np.sort(r_c,0)
            r_c = np.sort(r_c,1)
            l_c = np.round(l_c)
            r_c = np.round(r_c)
            l_c = l_c.astype(np.int32)
            r_c = r_c.astype(np.int32)
            res_s = ""
            for i in range(54):
                res_s = res_s + f"{l_c[i][0]} {l_c[i][1]} {r_c[i][0]} {r_c[i][1]}\n"
            
            print(res_s)
            open("C:/Users/klay/Downloads/mobile_tof-2/calibration/mathc.txt",'w').write(res_s)
            # print(corners2_left[:][0],corners2_right[:][0])
        else:
            print("Chessboard couldn't detected. Image pair: ", left_im, " and ", right_im)
            continue

    # print(left_imgpoints[0][0][0])
    image_size = gray_right.shape  # If you have no acceptable pair, you may have an error here.
    return [objpoints, left_imgpoints, right_imgpoints]

left_yml_file = "C:/Users/klay/Downloads/mobile_tof-2/calibration/stereo_R.yml"
right_yml_file = r"C:\\Users\\klay\Downloads\\mobile_tof-2\\calibration\\rs.yml"
right_img_dir  = "C:\\Users\\klay\\Downloads\\mobile_tof-2\\calibration\\data_rs_stereo"
left_img_dir = "C:\\Users\\klay\\Downloads\\mobile_tof-2\\calibration\\data_rs_stereo"
save_file = "C:\\Users\\klay\\Downloads\\mobile_tof-2\\calibration\\stereo_rs.yml"
stereo_calibrate(left_yml_file, right_yml_file, left_img_dir, "imgL", right_img_dir,"imgR", "png", save_file, 0.35)

156 356 210 298
156 356 210 299
156 356 211 299
156 356 211 300
156 357 211 300
157 357 211 300
176 357 225 301
176 357 225 301
177 357 226 301
177 377 226 314
177 378 226 314
177 378 226 314
197 378 240 315
197 378 240 315
197 378 241 315
197 378 241 316
198 378 241 316
198 378 242 316
218 399 255 329
218 399 256 329
218 399 256 330
218 399 256 330
219 399 257 330
219 399 257 330
239 399 271 331
239 399 271 331
239 399 271 331
239 420 271 344
240 420 272 345
240 420 272 345
260 420 286 345
260 420 286 345
260 420 286 345
261 421 287 346
261 421 287 346
261 421 288 346
281 440 301 360
282 441 302 360
282 441 302 360
282 441 302 360
283 441 303 360
283 442 303 361
303 442 317 361
304 443 317 361
304 443 317 361
304 461 318 375
305 461 318 375
305 462 319 375
325 462 332 376
326 463 333 376
326 463 333 376
327 464 333 376
327 465 334 376
327 465 334 376

149 358 207 297
150 358 207 297
150 358 207 298
151 358 207 298
151 358 208 299
152 358 208 300
168 358 221 300
168 358 221 301
169 358

In [27]:
# 深度图获取
import os
import glob
import numpy as np
import cv2 as cv2
import sys
import copy

def get_disparity_map(imgL, imgR):
    window_size = 7  # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
    left_matcher = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities= 10*16,  # max_disp has to be dividable by 16 f. E. HH 192, 256
        blockSize=window_size,
        P1=8 * 3 * window_size ,
        # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
        P2=32 * 3 * window_size,
        disp12MaxDiff=7,
        uniquenessRatio=10,
        speckleWindowSize=50,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_HH4
    )
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
    # FILTER Parameters
    lmbda = 80000
    sigma = 1.3
    visual_multiplier = 6

    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)

    wls_filter.setSigmaColor(sigma)
    displ = left_matcher.compute(imgL, imgR)  # .astype(np.float32)/16
    dispr = right_matcher.compute(imgR, imgL)  # .astype(np.float32)/16
    displ = np.int16(displ)
    dispr = np.int16(dispr)
    filteredImg = wls_filter.filter(displ, imgL, None, dispr)  # important to put "imgL" here!!!
    unnormalized_filteredImg = copy.deepcopy(filteredImg)
    filteredImg = np.clip(filteredImg,0,None)
    filteredImg = cv2.normalize(src=filteredImg, dst=filteredImg, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX)
    filteredImg = np.uint8(filteredImg)

    return unnormalized_filteredImg,filteredImg


stereo_calibration_file = "C:\\Users\\klay\\Downloads\\mobile_tof-2\\calibration\\stereo_rs.yml"

K1, D1, K2, D2, R, T, E, F, R1, R2, P1, P2, Q = load_stereo_coefficients(stereo_calibration_file)
left_dir = "C:\\Users\\klay\\Downloads\\mobile_tof-2\\calibration\\data_rs_stereo"
right_dir = "C:\\Users\\klay\\Downloads\\mobile_tof-2\\calibration\\data_rs_stereo"

left_images_path = glob.glob(left_dir + "/imgL*.png")
right_images_path = glob.glob(right_dir + "/imgR*.png")


for i,left_image_path in enumerate(left_images_path[:]):
    right_image_path = right_images_path[i]
    left_image  = cv2.imread(left_image_path)
    right_image = cv2.imread(right_image_path)

    height, width, channel = left_image.shape  # We will use the shape for remap

    # 校正图像
    leftMapX, leftMapY = cv2.initUndistortRectifyMap(K1, D1, R1, P1, (width, height), cv2.CV_32FC1)
    rightMapX, rightMapY = cv2.initUndistortRectifyMap(K2, D2, R2, P2, (width, height), cv2.CV_32FC1)
    left_rectified = cv2.remap(left_image, leftMapX, leftMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
    right_rectified = cv2.remap(right_image, rightMapX, rightMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
    gray_left = cv2.cvtColor(left_rectified, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_rectified, cv2.COLOR_BGR2GRAY)
    cv2.imshow("t",np.hstack([left_rectified,right_rectified]))
    unnormalized_disparity_image,normalized_disparity_image = get_disparity_map(gray_left, gray_right)  # Get the disparity map
    # unnormalized_disparity_image = np.clip(unnormalized_disparity_image,0,None)
    # 视差图投影到3D xyz
    unnormalized_disparity_image = unnormalized_disparity_image.astype(np.float32) / 16
    points_3d = cv2.reprojectImageTo3D(unnormalized_disparity_image, Q)

    # 过滤z轴坐标值,最大值不超过6
    depth_map = points_3d[:, :, -1]
    unnormalized_depth_map = depth_map.copy()
    # depth_map = np.clip(depth_map, -10, 0)
    depth_map = np.clip(depth_map, 0, 10)
    # print(min(depth_map))

    filename = left_image_path.split("\\")[-1]
    depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    # depth_map = depth_map.astype(np.uint8)
    depth_map = 255 - depth_map
    depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)

    # stereo_pair = np.hstack([left_rectified, right_rectified])
    # h, w = stereo_pair.shape[0:2]
    # num_lines = 20
    # for i in range(num_lines):
    #     cv2.line(stereo_pair, (0, h * i // num_lines), (w, h * i // num_lines), (0, 0, 255), 1)
    # cv2.imshow("depth",depth_map)
    cv2.waitKey(0)
